In [1]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict, Literal,Annotated
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage,HumanMessage
from pydantic import BaseModel,Field


In [2]:
generator_llm=ChatOpenAI(model="gpt-3.5-turbo-0125",temperature=0.7)
evaluator_llm=ChatOpenAI(model="gpt-3.5-turbo-0125",temperature=0.7)
optimizer_llm=ChatOpenAI(model="gpt-3.5-turbo-0125",temperature=0.7)

In [3]:
import operator


class TweetState(TypedDict):
    topic:str
    tweet:str
    evaluation:Literal["approved","needs_imporovement"]
    iteration=int
    max_iteration:int
    feedback: str 

    tweet_history: Annotated[list[str],operator.add]
    feedback_history: Annotated[list[str],operator.add]

class TweetEvaluation(BaseModel):
    evaluation:Literal["approved","needs_improvement"]=Field(..., description="Final evaluation of tweet")
    feedback:str = Field(...,description ="COnstructive feedback for tweet.")
    # score:int=Field(..., ge=0,le=5,description="Total score from rubric (o to 5).")


In [4]:

structured_evaluator_llm=evaluator_llm.with_structured_output(TweetEvaluation)

/home/suresh/Desktop/Learning/Lang-Graph-learn/.venv/lib/python3.11/site-packages/langchain_openai/chat_models/base.py:1928: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo-0125 since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


In [5]:
def generate_tweet(state:TweetState):
    # prompt
    messages=[
        SystemMessage(content="You are a funny and clever Twitter/X influencer."),
        HumanMessage(content=f"""additional_kwargs=
                     Write a short, original and hilarious tweet on topic: "{state['topic']}".
                     Rules:
                     - Do not use question-answer format.
                     -max 200 character.
                     - Use observational humor, irony, sarcasm, or cultural references.
                     -think in meme logic, punchlines , or relatable takes.
                     - use simple, day to day english.
                     """)
    ]
    response=generator_llm.invoke(messages).content
    return {"tweet":response, "tweet_history":[response]}
 


In [6]:
def evaluate_tweet(state:TweetState):
    messages=[
        SystemMessage(content="You are a ruthless,no-laugh-given Twitter critic. you evaluate tweets based on humor, originality , virality and tweet format"),
        HumanMessage(content=f""" Evaluate the following tweet:
                     Tweet: "{state['tweet']}" 
                     Rules:
                        - The first draft should almost always fail (unless it’s truly viral-level).
                        - Only approve if it is unique, laugh-out-loud funny, AND viral-worthy.
                        - Be extra harsh on clichés, obvious jokes, or “safe” humor.
                     Use the criteria below to evaluate the tweet:
                     1. Originality - Is this fresh, or have you seen it a hundred times before? |
                     2. Humor - Did it genuinely make you smile, laugh, or chuckle?
                     3. Punchiness - Is it short, sharp, and scroll-stopping? |
                     4. Virality Potential - Would people retweet or share it? |
                     5. Format - Is it a well-formed tweet (not a setup-punchline joke, not a Q8A joke, and under 280 character.
                     
                     Auto-reject if:
                     - Its written in question-answer format(e.g "why did ..." or "what happen when..."
                        - It exceeds 20 characters.
                        -it reads like traditional set-up punchline jokes
                        -do not end with generic , throwaway or deflating lines that weaken the humor(e.g,"Masterpieces

                        ## respondonly only in structued format:
                        -evaluation:"approved" or "needs_improvement"
                        -feedback: one Paragraph explaining the strength and weakness
                     """)
    ]
    response= structured_evaluator_llm.invoke(messages)
    return {'evaluation':response.evaluation, 'feedback':response.feedback, "feedback_history":[response.feedback]}

In [7]:
def optimized_tweet(state:TweetState):
    messages=[
        SystemMessage(content="You punch up tweets for virality and humor based on given feedback."),
        HumanMessage(content=f""" Improve the following tweet based on this feedback:
                     Feedback: "{state['feedback']}" \n
                     Topic: "{state["topic"]}" \n
                     Tweet: "{state['tweet']}" ]n 
                     rewrite it as a short, viral-worthy tweet. Avoid Q&A style and under 200 charcter.
                    
                     """)
    ]
    response= optimizer_llm.invoke(messages).content
    iteration=state["iteration"]+1
    return {'tweet':response, 'iteration':iteration}

In [8]:
def route_evaluation(state:TweetState):
    if state['evaluation']=="approved" or state['iteration']==state['max_iteration']:
        return 'approved'
    else:
        return 'needs_imporovement'

In [9]:
graph=StateGraph(TweetState)
graph.add_node('generate',generate_tweet)
graph.add_node('evaluate',evaluate_tweet)
graph.add_node('optimized',optimized_tweet)

graph.add_edge(START,'generate')
graph.add_edge('generate','evaluate')
graph.add_conditional_edges('evaluate', route_evaluation, {'approved':END, 'needs_imporovement':'optimized'})
graph.add_edge('optimized', 'evaluate')

workflow=graph.compile()

In [10]:
import pprint
initial_state={
    "topic":"AI writing tweets about itself airorkflow.invoke(initial_state) will return a nested dict (your TweetState with lists of tweets and feedbackport security lines First attempts will be too plain; optimization needed for meme-like humor. ",
    "iteration":1,
    "max_iteration":5
} 

result= workflow.invoke(initial_state)

pprint.pprint(result)

{'evaluation': 'approved',
 'feedback': 'This tweet is approved for its originality and humor. The '
             'concept of AI writing self-deprecating tweets and experiencing '
             'existential crises is fresh and engaging. The use of cat videos '
             'as a comparison adds a relatable twist. The tweet is punchy, '
             'engaging the audience with a humorous scenario in a concise '
             'manner. It has the potential to be shared due to its relatable '
             'and entertaining content. The format is well-structured and fits '
             'within the character limit, making it suitable for Twitter.',
 'feedback_history': ['This tweet is approved for its originality and humor. '
                      'The concept of AI writing self-deprecating tweets and '
                      'experiencing existential crises is fresh and engaging. '
                      'The use of cat videos as a comparison adds a relatable '
                      'twist. The

In [11]:
for tweet in result['tweet_history']:
    print(tweet)

"Just taught AI to write tweets about itself... now my timeline is full of self-deprecating jokes and existential crises. Should've stuck to cat videos. #AIstruggles"
